In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

Processing /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl


In [4]:
!pip install mordred --no-index --find-links=file:///kaggle/input/mordred-1-2-0-py3-none-any/

Looking in links: file:///kaggle/input/mordred-1-2-0-py3-none-any/
Processing /kaggle/input/mordred-1-2-0-py3-none-any/mordred-1.2.0-py3-none-any.whl
Processing /kaggle/input/mordred-1-2-0-py3-none-any/networkx-2.8.8-py3-none-any.whl (from mordred)
  Attempting uninstall: networkx
    Found existing installation: networkx 3.5
    Uninstalling networkx-3.5:
      Successfully uninstalled networkx-3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
nx-cugraph-cu12 25.2.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu

**Libraries**

In [5]:
import pandas as pd
import numpy as np
import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem
from mordred import Calculator, descriptors
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor

**Data loading and Preprocessing**

In [6]:
useless_cols = [   
    
    'MaxPartialCharge', 
    # Nan data
    'BCUT2D_MWHI',
    'BCUT2D_MWLOW',
    'BCUT2D_CHGHI',
    'BCUT2D_CHGLO',
    'BCUT2D_LOGPHI',
    'BCUT2D_LOGPLOW',
    'BCUT2D_MRHI',
    'BCUT2D_MRLOW',

    # Constant data
    'NumRadicalElectrons',
    'SMR_VSA8',
    'SlogP_VSA9',
    'fr_barbitur',
    'fr_benzodiazepine',
    'fr_dihydropyridine',
    'fr_epoxide',
    'fr_isothiocyan',
    'fr_lactam',
    'fr_nitroso',
    'fr_prisulfonamd',
    'fr_thiocyan',

    # High correlated data >0.95
    'MaxEStateIndex',
    'HeavyAtomMolWt',
    'ExactMolWt',
    'NumValenceElectrons',
    'Chi0',
    'Chi0n',
    'Chi0v',
    'Chi1',
    'Chi1n',
    'Chi1v',
    'Chi2n',
    'Kappa1',
    'LabuteASA',
    'HeavyAtomCount',
    'MolMR',
    'Chi3n',
    'BertzCT',
    'Chi2v',
    'Chi4n',
    'HallKierAlpha',
    'Chi3v',
    'Chi4v',
    'MinAbsPartialCharge',
    'MinPartialCharge',
    'MaxAbsPartialCharge',
    'FpDensityMorgan2',
    'FpDensityMorgan3',
    'Phi',
    'Kappa3',
    'fr_nitrile',
    'SlogP_VSA6',
    'NumAromaticCarbocycles',
    'NumAromaticRings',
    'fr_benzene',
    'VSA_EState6',
    'NOCount',
    'fr_C_O',
    'fr_C_O_noCOO',
    'NumHDonors',
    'fr_amide',
    'fr_Nhpyrrole',
    'fr_phenol',
    'fr_phenol_noOrthoHbond',
    'fr_COO2',
    'fr_halogen',
    'fr_diazo',
    'fr_nitro_arom',
    'fr_phos_ester'
]

In [7]:
test = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
tg = pd.read_csv('/kaggle/input/modred-dataset/desc_tg.csv',low_memory=False)
tc = pd.read_csv('/kaggle/input/modred-dataset/desc_tc.csv',low_memory=False)
rg = pd.read_csv('/kaggle/input/modred-dataset/desc_rg.csv',low_memory=False)
ffv = pd.read_csv('/kaggle/input/modred-dataset/desc_ffv.csv',low_memory=False)
density = pd.read_csv('/kaggle/input/modred-dataset/desc_de.csv',low_memory=False)
test = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv',low_memory=False)
Id= test['id']

In [8]:
for i in (tg,tc,rg,ffv,density):
     i.drop(columns=[col for col in i.columns if i[col].nunique() == 1],axis=1,inplace=True)

In [9]:
tg = tg.select_dtypes(exclude=['object', 'category'])
rg = rg.select_dtypes(exclude=['object', 'category'])
ffv = ffv.select_dtypes(exclude=['object', 'category'])
tc = tc.select_dtypes(exclude=['object', 'category'])
density  = density.select_dtypes(exclude=['object', 'category'])

In [10]:
test_mol = [Chem.MolFromSmiles(s) for s in test.SMILES]

In [11]:
calc = Calculator(descriptors, ignore_3D=True)
desc_test = calc.pandas(test_mol)

100%|██████████| 3/3 [00:00<00:00,  5.37it/s]


In [12]:
def make_smile_canonical(smile):
    try:
        mol = Chem.MolFromSmiles(smile)
        canon_smile = Chem.MolToSmiles(mol, canonical=True)
        return canon_smile
    except:
        return np.nan
test['SMILES'] = test['SMILES'].apply(lambda s: make_smile_canonical(s))

In [13]:
def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in useless_cols]
    descriptors = [compute_all_descriptors(smi) for smi in df['SMILES'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['SMILES']:
         compute_graph_features(smile, graph_feats)
        
    result = pd.concat(
        [
            pd.DataFrame(descriptors, columns=desc_names),
            pd.DataFrame(graph_feats)
        ],
        axis=1
    )

    result = result.replace([-np.inf, np.inf], np.nan)
    return result

In [14]:
def compute_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(desc_names)
    return [desc[1](mol) for desc in Descriptors.descList if desc[0] not in useless_cols]

def compute_graph_features(smiles, graph_feats):
    mol = Chem.MolFromSmiles(smiles)
    adj = rdmolops.GetAdjacencyMatrix(mol)
    G = nx.from_numpy_array(adj)

    graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
    graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
    graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

test = pd.concat([test, preprocessing(test)], axis=1)
test['Ipc']=np.log10(test['Ipc'])

test=test.drop(['id','SMILES'],axis=1)

**Model Training**

In [15]:
# !pip install optuna

In [16]:
# import optuna

In [17]:
# def objective(trial, x, y):
#     x = x.loc[:, x.nunique() > 1]
#     x = x.fillna(x.mean())
#     X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
    
#     n_estimators = trial.suggest_int('n_estimators', 200, 500)
#     learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1)
#     max_depth = trial.suggest_int('max_depth', 3, 15)
#     min_child_samples = trial.suggest_int('min_child_samples', 1, 20)
#     num_leaves = trial.suggest_int('num_leaves', 31, 150)
#     reg_lambda = trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True)
#     reg_alpha = trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True)
#     subsample = trial.suggest_float('subsample', 0.6, 1.0)
#     colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)
#     min_gain_to_split = trial.suggest_float('min_gain_to_split', 0.0, 1.0)
    
#     model = LGBMRegressor(
#         n_estimators=n_estimators,
#         learning_rate=learning_rate,
#         max_depth=max_depth,
#         min_child_samples=min_child_samples,
#         num_leaves=num_leaves,
#         reg_lambda=reg_lambda,
#         reg_alpha=reg_alpha,
#         subsample=subsample,
#         colsample_bytree=colsample_bytree,
#         min_gain_to_split=min_gain_to_split,
#         force_col_wise=True,
#         verbosity=-1,
#         random_state=42,
#         n_jobs=-1
#     )
    
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     mae = mean_absolute_error(y_test, y_pred)

#     return mae

In [18]:
train_cols1 = set(tg.columns) - {"Tg"}
test_cols1 = set(desc_test.columns)
common_cols1 = list(train_cols1 & test_cols1)
x_tg=tg[common_cols1].copy()
y_tg=tg["Tg"].copy()

In [19]:
X_tg_train, X_tg_test, y_tg_train, y_tg_test=train_test_split(x_tg, y_tg, test_size=0.1, random_state=10)

In [20]:
params_tg = {'n_estimators': 482,
             'learning_rate': 0.09072995964170064,
             'max_depth': 9,
             'min_child_samples': 1,
             'num_leaves': 82,
             'reg_lambda': 0.5307523215869501,
             'reg_alpha': 0.14507929585834298,
             'subsample': 0.9556450711830952,
             'colsample_bytree': 0.6327035143765806,
             'min_gain_to_split': 0.24961295883076395
            }

In [46]:
Model_tg_best = LGBMRegressor(
   **params_tg,
    force_col_wise=True,
    verbosity=-1,
    random_state=42,
    n_jobs=-1
)
Model_tg_best.fit(X_tg_train, y_tg_train)

LGBMRegressor(colsample_bytree=0.6327035143765806, force_col_wise=True,
              learning_rate=0.09072995964170064, max_depth=9,
              min_child_samples=1, min_gain_to_split=0.24961295883076395,
              n_estimators=482, n_jobs=-1, num_leaves=82, random_state=42,
              reg_alpha=0.14507929585834298, reg_lambda=0.5307523215869501,
              subsample=0.9556450711830952, verbosity=-1)

In [38]:
pred_tg = Model_tg_best.predict(X_tg_test)
print(mean_absolute_error(pred_tg, y_tg_test))
pred_train_tg = Model_tg_best.predict(X_tg_train)
print(mean_absolute_error(pred_train_tg, y_tg_train))

21.018065913123294
2.0719205783575436


In [39]:
train_cols2 = set(rg.columns) - {"Rg"}
test_cols2 = set(desc_test.columns)
common_cols2 = list(train_cols2 & test_cols2)
x_r=rg[common_cols2].copy()
y_r=rg["Rg"].copy()

In [40]:
X_rg_train, X_rg_test, y_rg_train, y_rg_test= train_test_split(x_r, y_r, test_size=0.1, random_state=10)

In [43]:
params_rg = {'n_estimators': 482,
             'learning_rate': 0.061641672923194904,
             'max_depth': 5,
             'min_child_samples': 9,
             'num_leaves': 109,
             'reg_lambda': 1.757306436103371,
             'reg_alpha': 5.197802924482619,
             'subsample': 0.980810284439084,
             'colsample_bytree': 0.8287520478021174,
             'min_gain_to_split': 0.843963391409551
            }

In [45]:
Model_rg_best = LGBMRegressor(
   **params_rg,
    force_col_wise=True,
    verbosity=-1,
    random_state=42,
    n_jobs=-1
)
Model_rg_best.fit(X_rg_train, y_rg_train)

LGBMRegressor(colsample_bytree=0.8287520478021174, force_col_wise=True,
              learning_rate=0.061641672923194904, max_depth=5,
              min_child_samples=9, min_gain_to_split=0.843963391409551,
              n_estimators=482, n_jobs=-1, num_leaves=109, random_state=42,
              reg_alpha=5.197802924482619, reg_lambda=1.757306436103371,
              subsample=0.980810284439084, verbosity=-1)

In [47]:
pred_rg = Model_rg_best.predict(X_rg_test)
print(mean_absolute_error(pred_rg, y_rg_test))
pred_train_rg = Model_rg_best.predict(X_rg_train)
print(mean_absolute_error(pred_train_rg, y_rg_train))

1.6822646879032013
0.6342589985189673


In [48]:
train_cols3 = set(ffv.columns) - {"FFV"}
test_cols3 = set(desc_test.columns)
common_cols3 = list(train_cols3 & test_cols3)
x_f=ffv[common_cols3].copy()
y_f=ffv["FFV"].copy()

In [49]:
X_f_train, X_f_test, y_f_train, y_f_test= train_test_split(x_f, y_f, test_size=0.1, random_state=10)

In [52]:
params_ffv = {'n_estimators': 300,
              'learning_rate': 0.05542231893239167,
              'max_depth': 11,
              'min_child_samples': 14,
              'num_leaves': 142,
              'reg_lambda': 0.001451017651983407,
              'reg_alpha': 0.03404925758492975,
              'subsample': 0.8436790853314469,
              'colsample_bytree': 0.7166238887424118,
              'min_gain_to_split': 0.0020082946075198694
             }

In [53]:
Model_ffv_best = LGBMRegressor(
   **params_ffv,
    force_col_wise=True,
    verbosity=-1,
    random_state=42,
    n_jobs=-1
)
Model_ffv_best.fit(X_f_train, y_f_train)

LGBMRegressor(colsample_bytree=0.7166238887424118, force_col_wise=True,
              learning_rate=0.05542231893239167, max_depth=11,
              min_child_samples=14, min_gain_to_split=0.0020082946075198694,
              n_estimators=300, n_jobs=-1, num_leaves=142, random_state=42,
              reg_alpha=0.03404925758492975, reg_lambda=0.001451017651983407,
              subsample=0.8436790853314469, verbosity=-1)

In [54]:
pred_f = Model_ffv_best.predict(X_f_test)
print(mean_absolute_error(pred_f, y_f_test))
pred_train_f = Model_ffv_best.predict(X_f_train)
print(mean_absolute_error(pred_train_f, y_f_train))

0.0073201265621939664
0.005768397030726988


In [55]:
train_cols4 = set(tc.columns) - {"Tc"}
test_cols4 = set(desc_test.columns)
common_cols4 = list(train_cols4 & test_cols4)
x_tc=tc[common_cols4].copy()
y_tc=tc["Tc"].copy()

In [56]:
x_t_tc, x_cv_tc, y_t_tc, y_cv_tc = train_test_split(x_tc, y_tc, test_size = 10, random_state = 42)

In [59]:
params_tc = {'n_estimators': 252,
             'learning_rate': 0.08928083735497692,
             'max_depth': 8,
             'min_child_samples': 11,
             'num_leaves': 89,
             'reg_lambda': 0.39495598235639107,
             'reg_alpha': 0.0012661260424439375,
             'subsample': 0.6932439887950764,
             'colsample_bytree': 0.9701628349361342,
             'min_gain_to_split': 0.0009777492782607339
            }

In [60]:
Model_tc_best = LGBMRegressor(
   **params_tc,
    force_col_wise=True,
    verbosity=-1,
    random_state=42,
    n_jobs=-1
)
Model_tc_best.fit(x_t_tc, y_t_tc)

LGBMRegressor(colsample_bytree=0.9701628349361342, force_col_wise=True,
              learning_rate=0.08928083735497692, max_depth=8,
              min_child_samples=11, min_gain_to_split=0.0009777492782607339,
              n_estimators=252, n_jobs=-1, num_leaves=89, random_state=42,
              reg_alpha=0.0012661260424439375, reg_lambda=0.39495598235639107,
              subsample=0.6932439887950764, verbosity=-1)

In [62]:
pred_tc = Model_tc_best.predict(x_cv_tc)
print(mean_absolute_error(pred_tc, y_cv_tc))
pred_train_tc = Model_tc_best.predict(x_t_tc)
print(mean_absolute_error(pred_train_tc, y_t_tc))

0.043917993249668344
0.01220372567611389


In [63]:
train_cols5 = set(density.columns) - {"Density"}
test_cols5 = set(desc_test.columns)
common_cols5 = list(train_cols5& test_cols5)
x_density=density[common_cols5].copy()
y_density=density["Density"].copy()

In [64]:
x_t_de, x_cv_de, y_t_de, y_cv_de = train_test_split(x_density, y_density, test_size = 10, random_state = 42)

In [67]:
params_density = {'n_estimators': 328,
                  'learning_rate': 0.0931491396758656,
                  'max_depth': 7,
                  'min_child_samples': 16,
                  'num_leaves': 53,
                  'reg_lambda': 0.0011323461947940542,
                  'reg_alpha': 0.5620336922210029,
                  'subsample': 0.7147749341041691,
                  'colsample_bytree': 0.6302066586866245,
                  'min_gain_to_split': 0.04286245475499195
                 }

In [68]:
Model_de_best = LGBMRegressor(
   **params_density,
    force_col_wise=True,
    verbosity=-1,
    random_state=42,
    n_jobs=-1
)
Model_de_best.fit(x_t_de, y_t_de)

LGBMRegressor(colsample_bytree=0.6302066586866245, force_col_wise=True,
              learning_rate=0.0931491396758656, max_depth=7,
              min_child_samples=16, min_gain_to_split=0.04286245475499195,
              n_estimators=328, n_jobs=-1, num_leaves=53, random_state=42,
              reg_alpha=0.5620336922210029, reg_lambda=0.0011323461947940542,
              subsample=0.7147749341041691, verbosity=-1)

In [69]:
pred_d = Model_de_best.predict(x_cv_de)
print(mean_absolute_error(pred_d, y_cv_de))
pred_train_de = Model_de_best.predict(x_t_de)
print(mean_absolute_error(pred_train_de, y_t_de))

0.055305564493856964
0.05646802573019902


**Prediction and Submission**

In [70]:
object_cols = desc_test.select_dtypes(include=['object']).columns
desc_test[object_cols] = desc_test[object_cols].apply(pd.to_numeric, errors='coerce')
desc_test.fillna(0, inplace=True)

In [72]:
sub = {'id': Id,
    'Tg':      Model_tg_best.predict(desc_test[common_cols1]),
    'FFV':     Model_ffv_best.predict(desc_test[common_cols3]),
    'Tc':      Model_tc_best.predict(desc_test[common_cols4]),
    'Density': Model_de_best.predict(desc_test[common_cols5]),
    'Rg':      Model_rg_best.predict(desc_test[common_cols2])
}

In [73]:
submission=pd.DataFrame(sub)

In [74]:
features = [col for col in submission.columns if col != 'id']

In [79]:
df_avg = submission.copy()
df_avg[features] = submission[features]
df_avg['Tg'] += 273.15

In [80]:
df_avg.to_csv('submission.csv',index=False)